In [1]:
import datetime as datetime
import datetime 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
events = pd.read_csv("events.csv.gzip", compression='gzip')
clicks = pd.read_csv("clicks.csv.gzip", compression='gzip')
installs = pd.read_csv("installs.csv.gzip", compression='gzip')
auctions = pd.read_csv("auctions.csv.gzip", compression='gzip')

pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

/home/maximiliano/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/maximiliano/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/maximiliano/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Transformo a datetime la fecha

In [3]:
installs['created'] = pd.to_datetime(installs['created'])
events['date'] = pd.to_datetime(events['date'])
clicks['created'] = pd.to_datetime(clicks['created'])

In [4]:
installs['date_seconds_level'] = installs['created'].astype('datetime64[s]')
events['date_seconds_level'] = events['date'].astype('datetime64[s]')

### Agrego diferencias entre installs de la misma aplicacion para el mismo usuario (para borrar posibles errores de la tracking platform luego)

In [5]:
installs['seconds_to_prev_install'] = installs.sort_values(['ref_hash', 'created'], ascending=True).groupby(['ref_hash', 'application_id'])['created'].diff()
installs['seconds_to_prev_install'] = installs['seconds_to_prev_install']/np.timedelta64(1, 's')

### Elimino de installs las instalaciones con menos de un segundo de diferencia para el mismo usuario y misma applicacion (las suponemos error de tracking platform) (22077 installs menos)

In [6]:
installs = installs.loc[(installs['seconds_to_prev_install'] > 1) | pd.isnull(installs['seconds_to_prev_install'])]

### Busco los installs que no se corresponden con un evento (un install deberia tener un evento en la tabla events para esa fecha). Para ellos genero un evento (774 eventos nuevos)

In [7]:
installs_with_events = pd.merge(events, installs, on=['ref_hash', 'date_seconds_level', 'application_id'])
installs_with_events['install_with_event'] = True
installs = pd.merge(installs, installs_with_events[['ref_hash', 'date_seconds_level', 'application_id', 'install_with_event']], on=['ref_hash', 'date_seconds_level', 'application_id'], how='left')
installs.head()
installs.loc[installs['install_with_event'] != True, 'install_with_event'] = False

In [8]:
#Creo el evento por cada install no correspondido (solo los installs implicitos tienen un evento correspondiente)
events = pd.concat([events, installs.loc[(installs['install_with_event'] == False) & (installs['implicit'] == True), ['ref_hash', 'created', 'application_id']]], ignore_index=True)

/home/maximiliano/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [9]:
events.loc[pd.isnull(events['date']), 'date'] = pd.to_datetime(events['created'])
events.drop(['created'], axis=1, inplace=True)

### Agrego columnas a events, clicks y a installs para poder clasificar despues

In [10]:
installs['install_day'] = installs['created'].dt.day
events['event_day'] = events['date'].dt.day
clicks['click_day'] = clicks['created'].dt.day

# Creando el dataframe

### Separo los dataframe para obtener las distintas ventanas con las cuales entrenar

In [11]:
print(installs['created'].min(), installs['created'].max())
print(events['date'].min(), events['date'].max())
print(clicks['created'].min(), clicks['created'].max())

2019-04-18 00:00:01.560000 2019-04-26 23:59:58.788000
2019-04-18 00:00:00.027000 2019-04-26 23:59:59.881000
2019-04-12 00:00:01.981000 2019-04-26 23:59:22.065000


In [12]:
df_installs = {}
df_events = {}
df_clicks = {}
devices = {}
for i in range(18, 23):
    windowStartDate = datetime.datetime(2019, 4, i)
    windowEndDate = datetime.datetime(2019, 4, i + 3)
    df_installs[i - 18] = installs.loc[(installs['created'] >= windowStartDate) & (windowEndDate >= installs['created'])]
    df_events[i - 18] = events.loc[(events['date'] >= windowStartDate) & (windowEndDate >= events['date'])]
    df_clicks[i - 18] = clicks.loc[(clicks['created'] >= windowStartDate) & (windowEndDate >= clicks['created'])]
    
    #Creo un dataframe de dispositivos con todos los dispositivos que tienen eventos para esa ventana
    devices[i - 18] = pd.DataFrame(df_events[i - 18]['ref_hash'].unique())
    
    #Agrego los segundos a la proxima conversion (desde el inicio de la ventana)
    next_window_installs = installs.loc[(installs['created'] >= datetime.datetime(2019, 4, i + 3)) & (datetime.datetime(2019, 4, i + 5) >= installs['created'])]
    devices[i - 18].columns = ['ref_hash']
    devices[i - 18] = pd.merge(devices[i - 18], next_window_installs.groupby('ref_hash')['created'].min().reset_index(), on='ref_hash', how='left')
    devices[i - 18].rename(columns = {'created':'seconds_to_conversion'}, inplace=True)
    devices[i - 18]['seconds_to_conversion'] = (devices[i - 18]['seconds_to_conversion'] - windowStartDate)/np.timedelta64(1, 's')
    #Completo los seconds_to_conversion de los que no convirtieron con el valor correspondiente a 3 dias en segundos
    devices[i - 18].loc[pd.isnull(devices[i - 18]['seconds_to_conversion']), 'seconds_to_conversion'] = 259200

## Agrego features para cada dataframe de devices

In [13]:
for i in range(0, 5):
    #Porcentaje de installs implicitos y no implicitos en el periodo + total installs en el periodo
    #devices[i] = pd.merge(devices[i], df_installs[i].groupby('ref_hash')['created'].count().reset_index(), on='ref_hash')
    #devices[i].rename(columns = {'created':'total_installs'}, inplace=True
    installs_by_implicit = pd.pivot_table(df_installs[i], index='ref_hash', columns='implicit', values='created', aggfunc='count', fill_value=0, margins=True).reset_index()
    installs_by_implicit.drop(installs_by_implicit.tail(1).index, inplace=True) #Elimino el margin creado por pivot_table
    installs_by_implicit.columns = ['ref_hash', 'installs_not_implicit', 'installs_implicit', 'total_installs']
    installs_by_implicit['ref_hash'] = installs_by_implicit['ref_hash'].astype('int64')
    installs_by_implicit['installs_not_implicit'] = installs_by_implicit['installs_not_implicit']/installs_by_implicit['total_installs']
    installs_by_implicit['installs_implicit'] = installs_by_implicit['installs_implicit']/installs_by_implicit['total_installs']
    devices[i] = pd.merge(devices[i], installs_by_implicit, on='ref_hash', how='left')
    
    #Porcentaje de installs por dia en el periodo
    installs_by_day = pd.pivot_table(df_installs[i], index='ref_hash', columns='install_day', values='created', aggfunc='count', fill_value=0, margins=True).reset_index()
    installs_by_day.drop(installs_by_day.tail(1).index, inplace=True) #Elimino el margin creado por pivot_table
    installs_by_day.columns = ['ref_hash', 'installs_day_1', 'installs_day_2', 'installs_day_3', 'total_installs']
    installs_by_day['ref_hash'] = installs_by_day['ref_hash'].astype('int64')
    installs_by_day['installs_day_1'] = installs_by_day['installs_day_1']/installs_by_day['total_installs']
    installs_by_day['installs_day_2'] = installs_by_day['installs_day_2']/installs_by_day['total_installs']
    installs_by_day['installs_day_3'] = installs_by_day['installs_day_3']/installs_by_day['total_installs']
    devices[i] = pd.merge(devices[i], installs_by_day[['ref_hash', 'installs_day_1', 'installs_day_2', 'installs_day_3']], on='ref_hash', how='left')
    
    #Porcentaje de eventos por dia en el periodo + total eventos
    #devices[i] = pd.merge(devices[i], df_events[i].groupby('ref_hash')['date'].count().reset_index(), on='ref_hash')
    #devices[i].rename(columns = {'date':'total_events'}, inplace=True)
    events_by_day = pd.pivot_table(df_events[i], index='ref_hash', columns='event_day', values='date', aggfunc='count', fill_value=0, margins=True).reset_index()
    events_by_day.drop(events_by_day.tail(1).index, inplace=True) #Elimino el margin creado por pivot_table
    events_by_day.columns = ['ref_hash', 'events_day_1', 'events_day_2', 'events_day_3', 'total_events']
    events_by_day['ref_hash'] = events_by_day['ref_hash'].astype('int64')
    events_by_day['events_day_1'] = events_by_day['events_day_1']/events_by_day['total_events']
    events_by_day['events_day_2'] = events_by_day['events_day_2']/events_by_day['total_events']
    events_by_day['events_day_3'] = events_by_day['events_day_3']/events_by_day['total_events']
    devices[i] = pd.merge(devices[i], events_by_day, on='ref_hash', how='left')
    
    #Calculo el porcentaje de eventos hechos a traves de wifi (esto puede influir en un futuro install)
    events_by_wifi = pd.pivot_table(df_events[i], index='ref_hash', columns='wifi', values='date', aggfunc='count', fill_value=0).reset_index()
    events_by_wifi['events_on_wifi'] = events_by_wifi[True]/(events_by_wifi[True] + events_by_wifi[False])
    devices[i] = pd.merge(devices[i], events_by_wifi[['ref_hash', 'events_on_wifi']], on='ref_hash', how='left')
    
    #Completo los nan con 0 en las columnas correspondientes
    devices[i][['installs_not_implicit', 'installs_implicit', 'total_installs', 'installs_day_1', 'installs_day_2', 'installs_day_3', 'total_events', 'events_day_1', 'events_day_2', 'events_day_3', 'events_on_wifi']].fillna(0)

## Agrego informacion del dispositivo. Esto puede tardar

In [15]:
def get_most_frequent_value(s):
    return np.nan if s.isnull().all() else s.value_counts().index[0]

for i in range(0, 5):
    #Agrego device_model y device_brand a los training sets
    info_group = df_events[i].groupby('ref_hash')
    devices_info = info_group['device_model'].apply(get_most_frequent_value).reset_index()
    #devices_info['device_brand'] = info_group['device_brand'].apply(get_most_frequent_value).reset_index()['device_brand']
    #devices_info['ref_type'] = info_group['ref_type'].apply(get_most_frequent_value).reset_index()['ref_type']
    #devices_info['device_os'] = info_group['device_os'].apply(get_most_frequent_value).reset_index()['device_os']
    devices[i] = pd.merge(devices[i], devices_info, on='ref_hash', how='left')

KeyboardInterrupt: 

In [21]:
devices[0]

,ref_hash,seconds_to_conversion,installs_not_implicit,installs_implicit,total_installs,installs_day_1,installs_day_2,installs_day_3,events_day_1,events_day_2,events_day_3,total_events,events_on_wifi,device_model,ref_type
0,5857744372586891366,259200.000,1.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,1.000,3,0.000,4318294190479584256.000,1891515180541284352.000
1,7642521036780133571,259200.000,nan,nan,nan,nan,nan,nan,0.011,0.189,0.800,95,0.000,2791160580186341888.000,1891515180541284352.000
2,2548841562898283198,259200.000,1.000,0.000,1.000,1.000,0.000,0.000,0.540,0.000,0.460,163,0.000,1658417010837625088.000,1891515180541284352.000
3,609402887625919085,259200.000,nan,nan,nan,nan,nan,nan,0.000,0.000,1.000,141,0.000,6298171771832984576.000,1891515180541284352.000
4,9114651763556439823,259200.000,nan,nan,nan,nan,nan,nan,0.000,0.350,0.650,60,0.000,3965274283132784640.000,1891515180541284352.000
5,3465901938095779347,259200.000,nan,nan,nan,nan,nan,nan,0.000,0.000,1.000,159,0.000,5992201963853464576.000,1891515180541284352.000
6,4048353602986107204,259200.000,nan,nan,nan,nan,nan,nan,0.228,0.693,0.079,189,0.000,5127787682281974784.000,1891515180541284352.000
7,221847198185594130,259200.000,1.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,1.000,3,0.000,3017401437185711104.000,1891515180541284352.000
8,2225835685291184910,259200.000,nan,nan,nan,nan,nan,nan,0.000,0.404,0.596,89,0.000,9186120447236368384.000,1891515180541284352.000
9,7624694062230398365,259200.000,1.000,0.000,1.000,1.000,0.000,0.000,0.155,0.492,0.353,323,0.000,3861844014453643264.000,1891515180541284352.000
